In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/csgrad/kaushik3/LLM/Kaggle_63LLMs/LLAMA/human_llama.csv')

In [3]:
df['source'].value_counts()

Human          278018
LLaMA-7B         7477
LLaMA-13B        7469
LLaMA-65B        7465
LLaMA-30B        7455
LLaMA-2-70B      3989
LLaMA-2-7B        331
Name: source, dtype: int64

In [4]:
AI = df[df['source'].str.startswith('LLaMA')]


In [7]:
AI.head()

,text,source,prompt_id,text_length,word_count,label_map
6,"Dear Mrs. Smith,\n\nI think that policy 1 is t...",LLaMA-2-70B,14,1449,269,1
41,Large and mysterious land carvings have been d...,LLaMA-65B,0,2456,412,1
46,The 30-year-old Scot was sent off after 20 min...,LLaMA-13B,0,2585,537,1
49,I'm in shopping heaven!! Usually I'm a little ...,LLaMA-7B,0,2030,405,1
52,Zoos have been a source of entertainment and e...,LLaMA-2-70B,53,2630,433,1


In [5]:
Human = df[df['source'] == 'Human']


In [6]:
Human = Human[:len(AI)]

In [7]:
print(len(Human))
print(len(AI))

34186
34186


In [8]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
torch.manual_seed(42)


In [9]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()
import numpy as np
def remove_outliers(embeddings):
        # Calculate the mode of the embeddings
        mode_val = np.mean(embeddings[0], axis=None)
        
        # Replace outliers with the mode value
        embeddings[np.abs(embeddings - mode_val) > 2 * np.std(embeddings)] = mode_val
        
        return embeddings
def min_max_normalize(embeddings):
    # Find the minimum and maximum values in the embeddings
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    
    # Normalize the embeddings to range [0, 255]
    normalized_embeddings = 255 * (embeddings - min_val) / (max_val - min_val)
    
    return normalized_embeddings.astype(np.uint8)  # Convert to uint8 for integer values between 0 and 255

def reshape_embeddings(embeddings):
    # Reshape the embeddings to 3D array
    return embeddings.reshape(32,32)

In [ ]:
AI['embeddings'] = AI['text'].apply(generate_bert_embeddings)
AI['embeddings1'] = AI['embeddings'].apply(remove_outliers)
AI['normalized_embeddings'] = AI['embeddings1'].apply(min_max_normalize)
AI['reshaped_embeddings'] = AI['normalized_embeddings'].apply(reshape_embeddings)

In [19]:
AI_embeddings = AI['embeddings']

In [20]:

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/llama_embeddings.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, AI_embeddings)
    

In [25]:
AI['reshaped_embeddings'].iloc[0]

array([[ 96,  93, 120, ..., 190, 173, 138],
       [ 94,  92, 105, ..., 170, 100, 119],
       [109, 165,  94, ..., 175, 147,  90],
       ...,
       [156,  98,  82, ..., 199,  55, 114],
       [ 97,  38, 150, ..., 147, 144, 152],
       [ 92, 117,  98, ..., 161,  68, 161]], dtype=uint8)

In [26]:
import matplotlib.pyplot as plt

for i in range(len(AI['reshaped_embeddings'])):
    plt.imshow(AI['reshaped_embeddings'].iloc[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/Llama/'+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


In [11]:
Human['embeddings'] = Human['text'].apply(generate_bert_embeddings)
Human['embeddings1'] = Human['embeddings'].apply(remove_outliers)
Human['normalized_embeddings'] = Human['embeddings1'].apply(min_max_normalize)
Human['reshaped_embeddings'] = Human['normalized_embeddings'].apply(reshape_embeddings)

In [12]:
human_embeddings = Human['embeddings']

In [15]:

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/human_embeddings.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, human_embeddings)
    

: 

In [14]:
import matplotlib.pyplot as plt
for i in range(len(Human['reshaped_embeddings'])):
    plt.imshow(Human['reshaped_embeddings'].iloc[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/Human/'+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()